# 1. Introduction

Phoneix is becoming a public city for people to retire. Although it is hot during summer time, the living cost is well balanced. Recently, Alex and Jenny moved from Chicago to Phoenix and are looking for a place to open a restaurant. They would like to know what type of food they want to server. Who are the targeting customers and what is the price range? They also want to pick up a location where there are not having too many competetions but also with reasonable rent.

# 2. Data Source

The data from Foursquare data location will be used for the recommanding system. Data such as different type of restaurents, traffics, surroundings, etc. will be extracted and used for analysis. In addition, the population distribution and income levels from third party will also be used if those are not available on Foursquare. 

In [1]:
import pandas as pd
from geopy.geocoders import Nominatim

import requests
from bs4 import BeautifulSoup

res = requests.get("https://phoenix.areaconnect.com/zip2.htm?search=zip")
soup = BeautifulSoup(res.content,'html.parser')
#print(soup)
zipcode_list = []
zipcode = soup.find_all(class_ = 'block zip1')
for i in zipcode:
    zipcode_list.append(list(i.b)[0].strip())
df_zip =  pd.DataFrame(zipcode_list, columns = ['zipcode'])
latitude_list = []
latitude = soup.find_all(class_ = 'block zip6')
#print(list(latitude[0])[0])
for i in latitude:
    latitude_list.append(list(i)[0].strip())
df_zip['Latitude'] =  pd.DataFrame(latitude_list)
longitude_list = []
longitude = soup.find_all(class_ = 'block zip7')
for i in longitude:
    longitude_list.append(list(i)[0].strip())
df_zip['Longitude'] =  pd.DataFrame(longitude_list)
print(df_zip.head())

  zipcode Latitude Longitude
0   85001   33.449  -112.069
1   85002   33.451  -112.068
2   85003   33.452  -112.075
3   85004   33.451  -112.068
4   85005   33.448  -112.128


In [2]:
res_pop = requests.get("https://www.zipdatamaps.com/zipcodes-phoenix-az")
soup_pop = BeautifulSoup(res_pop.content,'html.parser')
#print(soup_pop)
pop_list = []
pop = soup_pop.find_all('table')[2]
#print(type(pop))
df_pop = pd.read_html(str(pop))[0]
df_pop = df_pop['List of Zipcodes in Phoenix, Arizona']
df_pop = df_pop.loc[df_pop['ZIP Code Name'] == 'Phoenix'].drop(['ZIP Code Name','Type'], axis = 1)
df_pop['ZIP Code'] = df_pop['ZIP Code'].astype('int').astype('str')
df_pop.rename(columns={'ZIP Code': "zipcode"},inplace=True)
df_pop.set_index('zipcode',inplace=True)
#print(df_pop)
df_zip.set_index('zipcode',inplace=True)
df = pd.concat([df_pop, df_zip], axis=1, join='inner')
df.reset_index(inplace=True)
print(df)

   zipcode  Population Latitude Longitude
0    85003      9369.0   33.452  -112.075
1    85004      4965.0   33.451  -112.068
2    85006     25742.0   33.469  -112.046
3    85007     14040.0   33.448  -112.090
4    85008     56145.0   33.470  -111.999
5    85009     52520.0   33.457  -112.123
6    85012      6390.0   33.513  -112.066
7    85013     19314.0   33.510  -112.080
8    85014     24782.0   33.514  -112.053
9    85015     37644.0   33.508  -112.101
10   85016     33896.0   33.504  -112.029
11   85017     38872.0   33.511  -112.122
12   85018     36065.0   33.503  -111.983
13   85019     25042.0   33.513  -112.142
14   85020     32850.0   33.559  -112.056
15   85021     37743.0   33.559  -112.089
16   85022     46395.0   33.628  -112.052
17   85023     31274.0   33.632  -112.112
18   85024     22898.0   33.688  -112.050
19   85027     37140.0   33.690  -112.107
20   85028     19157.0   33.580  -112.006
21   85029     43233.0   33.597  -112.118
22   85031     30493.0   33.495  -

In [3]:
address = 'Phoenix, Arizona'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Phoenix, Arizona are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Phoenix, Arizona are 33.4484367, -112.0741417.


In [4]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium
map_phoenix = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng in zip(df['Latitude'], df['Longitude']):
    #label = '{}, {}'.format(neighborhood, borough)
    #label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lng)],
        radius=5,
        #popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_phoenix)  
    
map_phoenix

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    branca-0.4.0               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be 

In [5]:
CLIENT_ID = '1FUXGNW4X0BTEOL4EGV5P0ILSBALJA2YOBDN4CWGFDORF5FK' # your Foursquare ID
CLIENT_SECRET = 'JIOP2NY1XPSUTJBMSOI2DOP0XIKBO3ZRMIDCECWPYH445KQV' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1FUXGNW4X0BTEOL4EGV5P0ILSBALJA2YOBDN4CWGFDORF5FK
CLIENT_SECRET:JIOP2NY1XPSUTJBMSOI2DOP0XIKBO3ZRMIDCECWPYH445KQV


In [6]:
zipcode_latitude = df.loc[0, 'Latitude'] # neighborhood latitude value
zipcode_longitude = df.loc[0, 'Longitude'] # neighborhood longitude value
zipcode_name = df.loc[0, 'zipcode'] # neighborhood longitude value

print('Latitude and longitude values of {} are {}, {}.'.format(zipcode_name, 
                                                               zipcode_latitude, 
                                                               zipcode_longitude))

Latitude and longitude values of 85003 are 33.452, -112.075.


In [7]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    zipcode_latitude, 
    zipcode_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=1FUXGNW4X0BTEOL4EGV5P0ILSBALJA2YOBDN4CWGFDORF5FK&client_secret=JIOP2NY1XPSUTJBMSOI2DOP0XIKBO3ZRMIDCECWPYH445KQV&v=20180605&ll=33.452,-112.075&radius=500&limit=100'

In [8]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e4cb09b923935001b5d314b'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Copper Square',
  'headerFullLocation': 'Copper Square, Phoenix',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 67,
  'suggestedBounds': {'ne': {'lat': 33.4565000045, 'lng': -112.06961663129468},
   'sw': {'lat': 33.447499995499996, 'lng': -112.08038336870533}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e3a4cdf1850f0453401b5fd',
       'name': 'Crescent Ballroom',
       'location': {'address': '308 N 2nd Ave',
        'crossStreet': 'at W Van Buren St',
        'lat': 33.451657,
        'lng': -112.076813

In [9]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [10]:
from pandas.io.json import json_normalize

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Crescent Ballroom,Rock Club,33.451657,-112.076813
1,Valley Bar,Music Venue,33.450120,-112.074103
2,Cornish Pasty Co,Pub,33.450226,-112.074109
3,The Westin Phoenix Downtown,Hotel,33.451669,-112.073028
4,Civic Space Park,Park,33.453520,-112.074459


In [11]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [12]:
phoenix_venues = getNearbyVenues(names=df['zipcode'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

85003
85004
85006
85007
85008
85009
85012
85013
85014
85015
85016
85017
85018
85019
85020
85021
85022
85023
85024
85027
85028
85029
85031
85032
85033
85034
85035
85037
85040
85041
85042
85043
85044
85045
85048
85050
85051
85053
85054
85085
85086


In [13]:
print(phoenix_venues.shape)
phoenix_venues = phoenix_venues[phoenix_venues['Venue Category'].str.contains('Restaurant')].reset_index(drop=True)
phoenix_venues

(3892, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,85003,33.452,-112.075,Phoenix Public Market Café,33.455692,-112.073086,New American Restaurant
1,85003,33.452,-112.075,The Breadfruit,33.455662,-112.072009,Caribbean Restaurant
2,85003,33.452,-112.075,The Vig Fillmore,33.454926,-112.079308,American Restaurant
3,85003,33.452,-112.075,Harumi Sushi,33.449785,-112.075276,Sushi Restaurant
4,85003,33.452,-112.075,The Arrogant Butcher,33.447354,-112.072632,American Restaurant
5,85003,33.452,-112.075,Chick-fil-A,33.453619,-112.071545,Fast Food Restaurant
6,85003,33.452,-112.075,Pita Jungle,33.458867,-112.077528,Mediterranean Restaurant
7,85003,33.452,-112.075,Province Urban Kitchen & Bar,33.451630,-112.073461,American Restaurant
8,85003,33.452,-112.075,Blue Hound Kitchen & Cocktails,33.447447,-112.073317,New American Restaurant
9,85003,33.452,-112.075,FEZ,33.459779,-112.074182,American Restaurant


In [14]:
phoenix_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
85003,15,15,15,15,15,15
85004,17,17,17,17,17,17
85006,31,31,31,31,31,31
85007,17,17,17,17,17,17
85008,31,31,31,31,31,31
85009,23,23,23,23,23,23
85012,30,30,30,30,30,30
85013,33,33,33,33,33,33
85014,31,31,31,31,31,31


In [15]:
print('There are {} uniques categories.'.format(len(phoenix_venues['Venue Category'].unique())))

There are 36 uniques categories.


In [16]:
phoenix_onehot = pd.get_dummies(phoenix_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
phoenix_onehot['Neighborhood'] = phoenix_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [phoenix_onehot.columns[-1]] + list(phoenix_onehot.columns[:-1])
phoenix_onehot = phoenix_onehot[fixed_columns]

phoenix_onehot.head()

,Neighborhood,African Restaurant,American Restaurant,Asian Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant,Eastern European Restaurant,Fast Food Restaurant,...,Peruvian Restaurant,Restaurant,Seafood Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Tex-Mex Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,85003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,85003,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,85003,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,85003,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,85003,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
phoenix_onehot.shape

(984, 37)

In [18]:
phoenix_grouped = phoenix_onehot.groupby('Neighborhood').mean().reset_index()
phoenix_grouped

,Neighborhood,African Restaurant,American Restaurant,Asian Restaurant,Caribbean Restaurant,Chinese Restaurant,Comfort Food Restaurant,Cuban Restaurant,Eastern European Restaurant,Fast Food Restaurant,...,Peruvian Restaurant,Restaurant,Seafood Restaurant,Southern / Soul Food Restaurant,Spanish Restaurant,Sushi Restaurant,Tex-Mex Restaurant,Thai Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,85003,0.000000,0.266667,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000,0.066667,...,0.000000,0.000000,0.000000,0.066667,0.000000,0.066667,0.000000,0.000000,0.000000,0.000000
1,85004,0.000000,0.352941,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.058824,...,0.000000,0.000000,0.000000,0.117647,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000
2,85006,0.000000,0.096774,0.000000,0.032258,0.032258,0.000000,0.000000,0.000000,0.000000,...,0.032258,0.096774,0.000000,0.064516,0.000000,0.000000,0.000000,0.064516,0.064516,0.000000
3,85007,0.000000,0.235294,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000,0.058824,...,0.000000,0.000000,0.000000,0.117647,0.000000,0.058824,0.000000,0.000000,0.000000,0.000000
4,85008,0.000000,0.129032,0.000000,0.000000,0.032258,0.032258,0.000000,0.000000,0.064516,...,0.000000,0.096774,0.064516,0.000000,0.000000,0.032258,0.000000,0.032258,0.000000,0.000000
5,85009,0.000000,0.130435,0.000000,0.086957,0.000000,0.000000,0.000000,0.000000,0.043478,...,0.000000,0.000000,0.000000,0.043478,0.000000,0.043478,0.000000,0.000000,0.000000,0.000000
6,85012,0.000000,0.133333,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000,0.033333,...,0.000000,0.033333,0.000000,0.033333,0.000000,0.100000,0.000000,0.033333,0.033333,0.000000
7,85013,0.000000,0.060606,0.030303,0.000000,0.030303,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.060606,0.000000,0.000000,0.000000,0.060606,0.000000,0.030303,0.030303,0.060606
8,85014,0.000000,0.193548,0.032258,0.000000,0.032258,0.000000,0.000000,0.000000,0.032258,...,0.000000,0.000000,0.000000,0.032258,0.000000,0.096774,0.000000,0.000000,0.032258,0.000000
9,85015,0.000000,0.148148,0.037037,0.000000,0.037037,0.000000,0.000000,0.000000,0.037037,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.037037,0.000000,0.037037,0.037037,0.074074


In [19]:
num_top_venues = 5

for hood in phoenix_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = phoenix_grouped[phoenix_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----85003----
                      venue  freq
0       American Restaurant  0.27
1   New American Restaurant  0.20
2        Mexican Restaurant  0.13
3          Sushi Restaurant  0.07
4  Mediterranean Restaurant  0.07


----85004----
                             venue  freq
0              American Restaurant  0.35
1          New American Restaurant  0.18
2  Southern / Soul Food Restaurant  0.12
3             Caribbean Restaurant  0.06
4                 Sushi Restaurant  0.06


----85006----
                           venue  freq
0             Mexican Restaurant  0.26
1        New American Restaurant  0.13
2            American Restaurant  0.10
3                     Restaurant  0.10
4  Vegetarian / Vegan Restaurant  0.06


----85007----
                             venue  freq
0              American Restaurant  0.24
1               Mexican Restaurant  0.24
2          New American Restaurant  0.18
3  Southern / Soul Food Restaurant  0.12
4                 Sushi Restaurant  0.06


----85

In [20]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [21]:
import numpy as np
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = phoenix_grouped['Neighborhood']

for ind in np.arange(phoenix_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(phoenix_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,85003,American Restaurant,New American Restaurant,Mexican Restaurant,Mediterranean Restaurant,Sushi Restaurant,Caribbean Restaurant,Southern / Soul Food Restaurant,Japanese Restaurant,Fast Food Restaurant,Vietnamese Restaurant
1,85004,American Restaurant,New American Restaurant,Southern / Soul Food Restaurant,Mediterranean Restaurant,Sushi Restaurant,Caribbean Restaurant,Japanese Restaurant,Fast Food Restaurant,Mexican Restaurant,Vietnamese Restaurant
2,85006,Mexican Restaurant,New American Restaurant,American Restaurant,Restaurant,Southern / Soul Food Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant,Thai Restaurant,Chinese Restaurant,Caribbean Restaurant
3,85007,Mexican Restaurant,American Restaurant,New American Restaurant,Southern / Soul Food Restaurant,Sushi Restaurant,Caribbean Restaurant,Mediterranean Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Greek Restaurant
4,85008,Mexican Restaurant,Italian Restaurant,American Restaurant,Restaurant,Fast Food Restaurant,Seafood Restaurant,Comfort Food Restaurant,Indian Restaurant,Mediterranean Restaurant,Halal Restaurant


In [22]:
from sklearn.cluster import KMeans

kclusters = 5

phoenix_grouped_clustering = phoenix_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(phoenix_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 1, 3, 1, 1, 3, 1, 3, 1], dtype=int32)

In [23]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

phoenix_merged = df
phoenix_merged.rename(columns={'zipcode': 'Neighborhood'}, inplace=True)

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
phoenix_merged = phoenix_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

phoenix_merged.head() # check the last columns!

,Neighborhood,Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,85003,9369.0,33.452,-112.075,3,American Restaurant,New American Restaurant,Mexican Restaurant,Mediterranean Restaurant,Sushi Restaurant,Caribbean Restaurant,Southern / Soul Food Restaurant,Japanese Restaurant,Fast Food Restaurant,Vietnamese Restaurant
1,85004,4965.0,33.451,-112.068,3,American Restaurant,New American Restaurant,Southern / Soul Food Restaurant,Mediterranean Restaurant,Sushi Restaurant,Caribbean Restaurant,Japanese Restaurant,Fast Food Restaurant,Mexican Restaurant,Vietnamese Restaurant
2,85006,25742.0,33.469,-112.046,1,Mexican Restaurant,New American Restaurant,American Restaurant,Restaurant,Southern / Soul Food Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant,Thai Restaurant,Chinese Restaurant,Caribbean Restaurant
3,85007,14040.0,33.448,-112.090,3,Mexican Restaurant,American Restaurant,New American Restaurant,Southern / Soul Food Restaurant,Sushi Restaurant,Caribbean Restaurant,Mediterranean Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Greek Restaurant
4,85008,56145.0,33.470,-111.999,1,Mexican Restaurant,Italian Restaurant,American Restaurant,Restaurant,Fast Food Restaurant,Seafood Restaurant,Comfort Food Restaurant,Indian Restaurant,Mediterranean Restaurant,Halal Restaurant


In [24]:
# create map
import matplotlib.cm as cm
import matplotlib.colors as colors
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(phoenix_merged['Latitude'], phoenix_merged['Longitude'], phoenix_merged['Neighborhood'], phoenix_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    #print(type(cluster))
    if str(cluster) == "nan":
        cluster = 5
    folium.CircleMarker(
        [float(lat), float(lon)],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [25]:
phoenix_merged.loc[phoenix_merged['Cluster Labels'] == 0, phoenix_merged.columns[[1,2] + list(range(3, phoenix_merged.shape[1]))]]

,Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,22898.0,33.688,-112.050,0,Fast Food Restaurant,Mexican Restaurant,American Restaurant,Vietnamese Restaurant,Persian Restaurant,Chinese Restaurant,Greek Restaurant,Mediterranean Restaurant,Italian Restaurant,Sushi Restaurant
29,54947.0,33.385,-112.103,0,Fast Food Restaurant,Mexican Restaurant,Seafood Restaurant,Asian Restaurant,Vietnamese Restaurant,Fondue Restaurant,Hawaiian Restaurant,Halal Restaurant,Greek Restaurant,Gluten-free Restaurant
31,30560.0,33.439,-112.191,0,Fast Food Restaurant,Mexican Restaurant,Chinese Restaurant,Thai Restaurant,Japanese Restaurant,Seafood Restaurant,Vietnamese Restaurant,Fondue Restaurant,Halal Restaurant,Greek Restaurant
39,17517.0,33.757,-112.131,0,Fast Food Restaurant,Asian Restaurant,Mexican Restaurant,Vietnamese Restaurant,American Restaurant,Tex-Mex Restaurant,Mediterranean Restaurant,German Restaurant,Halal Restaurant,Greek Restaurant


In [26]:
phoenix_merged.loc[phoenix_merged['Cluster Labels'] == 1, phoenix_merged.columns[[1,2] + list(range(3, phoenix_merged.shape[1]))]]

,Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,25742.0,33.469,-112.046,1,Mexican Restaurant,New American Restaurant,American Restaurant,Restaurant,Southern / Soul Food Restaurant,Vegetarian / Vegan Restaurant,Italian Restaurant,Thai Restaurant,Chinese Restaurant,Caribbean Restaurant
4,56145.0,33.470,-111.999,1,Mexican Restaurant,Italian Restaurant,American Restaurant,Restaurant,Fast Food Restaurant,Seafood Restaurant,Comfort Food Restaurant,Indian Restaurant,Mediterranean Restaurant,Halal Restaurant
5,52520.0,33.457,-112.123,1,Mexican Restaurant,American Restaurant,New American Restaurant,Italian Restaurant,Caribbean Restaurant,Sushi Restaurant,Southern / Soul Food Restaurant,Mediterranean Restaurant,Fast Food Restaurant,Greek Restaurant
7,19314.0,33.510,-112.080,1,Mexican Restaurant,Italian Restaurant,New American Restaurant,Restaurant,American Restaurant,Latin American Restaurant,Vietnamese Restaurant,Sushi Restaurant,Hawaiian Restaurant,Gluten-free Restaurant
9,37644.0,33.508,-112.101,1,Mexican Restaurant,American Restaurant,Vietnamese Restaurant,New American Restaurant,Italian Restaurant,Mediterranean Restaurant,Japanese Restaurant,Thai Restaurant,Asian Restaurant,Hawaiian Restaurant
11,38872.0,33.511,-112.122,1,Mexican Restaurant,Vietnamese Restaurant,Thai Restaurant,American Restaurant,Chinese Restaurant,New American Restaurant,Asian Restaurant,Caribbean Restaurant,Fast Food Restaurant,Gluten-free Restaurant
13,25042.0,33.513,-112.142,1,Mexican Restaurant,Vietnamese Restaurant,Chinese Restaurant,Italian Restaurant,Sushi Restaurant,Greek Restaurant,American Restaurant,Caribbean Restaurant,Thai Restaurant,Eastern European Restaurant
14,32850.0,33.559,-112.056,1,Mexican Restaurant,American Restaurant,Italian Restaurant,Latin American Restaurant,Chinese Restaurant,Greek Restaurant,Hotpot Restaurant,Vegetarian / Vegan Restaurant,Japanese Restaurant,Mediterranean Restaurant
15,37743.0,33.559,-112.089,1,Mexican Restaurant,Italian Restaurant,American Restaurant,Japanese Restaurant,Latin American Restaurant,Seafood Restaurant,Greek Restaurant,Vegetarian / Vegan Restaurant,Korean Restaurant,Fast Food Restaurant
16,46395.0,33.628,-112.052,1,Mexican Restaurant,Asian Restaurant,Thai Restaurant,Chinese Restaurant,Seafood Restaurant,Vietnamese Restaurant,Middle Eastern Restaurant,American Restaurant,Comfort Food Restaurant,Fast Food Restaurant


In [27]:
phoenix_merged.loc[phoenix_merged['Cluster Labels'] == 2, phoenix_merged.columns[[1,2] + list(range(3, phoenix_merged.shape[1]))]]

,Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
33,7238.0,33.304,-112.114,2,Asian Restaurant,Vietnamese Restaurant,Fondue Restaurant,Hotpot Restaurant,Hawaiian Restaurant,Halal Restaurant,Greek Restaurant,Gluten-free Restaurant,German Restaurant,Fast Food Restaurant


In [28]:
phoenix_merged.loc[phoenix_merged['Cluster Labels'] == 3, phoenix_merged.columns[[1,2] + list(range(3, phoenix_merged.shape[1]))]]

,Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,9369.0,33.452,-112.075,3,American Restaurant,New American Restaurant,Mexican Restaurant,Mediterranean Restaurant,Sushi Restaurant,Caribbean Restaurant,Southern / Soul Food Restaurant,Japanese Restaurant,Fast Food Restaurant,Vietnamese Restaurant
1,4965.0,33.451,-112.068,3,American Restaurant,New American Restaurant,Southern / Soul Food Restaurant,Mediterranean Restaurant,Sushi Restaurant,Caribbean Restaurant,Japanese Restaurant,Fast Food Restaurant,Mexican Restaurant,Vietnamese Restaurant
3,14040.0,33.448,-112.090,3,Mexican Restaurant,American Restaurant,New American Restaurant,Southern / Soul Food Restaurant,Sushi Restaurant,Caribbean Restaurant,Mediterranean Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Greek Restaurant
6,6390.0,33.513,-112.066,3,Mexican Restaurant,American Restaurant,Sushi Restaurant,New American Restaurant,Latin American Restaurant,Mediterranean Restaurant,Gluten-free Restaurant,Hawaiian Restaurant,Fast Food Restaurant,Vegetarian / Vegan Restaurant
8,24782.0,33.514,-112.053,3,Mexican Restaurant,American Restaurant,New American Restaurant,Sushi Restaurant,Latin American Restaurant,Mediterranean Restaurant,Italian Restaurant,Chinese Restaurant,Fast Food Restaurant,Southern / Soul Food Restaurant
10,33896.0,33.504,-112.029,3,American Restaurant,New American Restaurant,Mexican Restaurant,Italian Restaurant,Sushi Restaurant,Mediterranean Restaurant,Fast Food Restaurant,Thai Restaurant,Seafood Restaurant,Chinese Restaurant
12,36065.0,33.503,-111.983,3,American Restaurant,New American Restaurant,Seafood Restaurant,Mexican Restaurant,Italian Restaurant,Sushi Restaurant,Mediterranean Restaurant,Restaurant,Gluten-free Restaurant,Fast Food Restaurant
19,37140.0,33.690,-112.107,3,American Restaurant,Fast Food Restaurant,Mexican Restaurant,Chinese Restaurant,Greek Restaurant,Mediterranean Restaurant,Italian Restaurant,Sushi Restaurant,Asian Restaurant,Vegetarian / Vegan Restaurant
32,37993.0,33.324,-112.001,3,Mexican Restaurant,American Restaurant,Italian Restaurant,Sushi Restaurant,Indian Restaurant,Vegetarian / Vegan Restaurant,Seafood Restaurant,Restaurant,Mediterranean Restaurant,Fast Food Restaurant
35,25736.0,33.694,-112.000,3,American Restaurant,Restaurant,Mexican Restaurant,Sushi Restaurant,Fast Food Restaurant,Vietnamese Restaurant,Japanese Restaurant,Seafood Restaurant,Korean Restaurant,Mediterranean Restaurant


In [29]:
phoenix_merged.loc[phoenix_merged['Cluster Labels'] == 4, phoenix_merged.columns[[1,2] + list(range(3, phoenix_merged.shape[1]))]]

,Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,30493.0,33.495,-112.171,4,Mexican Restaurant,Chinese Restaurant,Fast Food Restaurant,Seafood Restaurant,Vietnamese Restaurant,Restaurant,German Restaurant,Japanese Restaurant,Italian Restaurant,African Restaurant
24,53037.0,33.491,-112.214,4,Mexican Restaurant,Fast Food Restaurant,Seafood Restaurant,Chinese Restaurant,Vietnamese Restaurant,Restaurant,Italian Restaurant,Thai Restaurant,Tex-Mex Restaurant,Southern / Soul Food Restaurant
26,47486.0,33.474,-112.188,4,Mexican Restaurant,Fast Food Restaurant,Chinese Restaurant,Seafood Restaurant,Sushi Restaurant,Vietnamese Restaurant,Italian Restaurant,Southern / Soul Food Restaurant,Halal Restaurant,American Restaurant
28,29352.0,33.409,-112.026,4,Mexican Restaurant,Chinese Restaurant,American Restaurant,Seafood Restaurant,Fast Food Restaurant,Thai Restaurant,Asian Restaurant,Restaurant,New American Restaurant,Mediterranean Restaurant
30,40962.0,33.375,-112.032,4,Mexican Restaurant,Chinese Restaurant,Fast Food Restaurant,Seafood Restaurant,Hawaiian Restaurant,Thai Restaurant,Asian Restaurant,New American Restaurant,Halal Restaurant,Greek Restaurant
34,33449.0,33.331,-112.047,4,Fast Food Restaurant,Chinese Restaurant,Seafood Restaurant,Restaurant,Mexican Restaurant,Fondue Restaurant,Vietnamese Restaurant,Halal Restaurant,Greek Restaurant,Gluten-free Restaurant
40,40495.0,33.837,-112.089,4,Mexican Restaurant,Fast Food Restaurant,American Restaurant,Fondue Restaurant,Hawaiian Restaurant,Halal Restaurant,Greek Restaurant,Gluten-free Restaurant,German Restaurant,Vietnamese Restaurant


In [30]:
phoenix_merged.loc[phoenix_merged['Cluster Labels'] == 5, phoenix_merged.columns[[1,2] + list(range(3, phoenix_merged.shape[1]))]]

,Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue


In [211]:
phoenix_merged.loc[phoenix_merged['Cluster Labels'] == 6, phoenix_merged.columns[[1,2] + list(range(3, phoenix_merged.shape[1]))]]

,Population,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
